In [1]:
import pandas as pd
import duckdb
from rich import print as rprint
import logging
import sys
import sqlite3

In [2]:
filepath = "/teamspace/studios/Data_Studio/product_listing/product_listing.csv"
data = pd.read_csv(filepath)
data.columns = ['Product_Name', 'Price', 'Rating', 'Description', 'Features']
rprint(data.columns)

Index(['Product_Name', 'Price', 'Rating', 'Description', 'Features'], dtype='object')

In [46]:
data.tail(1)

,Product_Name,Price,Rating,Description,Features
11,Home Gym Resistance Bands Set,49.99,4.3,Get a full-body workout at home with our Home ...,- Set includes multiple bands with different r...


In [4]:
data.loc[data.Price == 79.99, :]

,Product_Name,Price,Rating,Description,Features
0,Ultimate Wireless Bluetooth Earbuds,79.99,4.4,Elevate your music experience with our Ultimat...,- High-fidelity sound with deep bass and clear...
3,Portable Solar Charger,79.99,4.0,Stay charged on the go with our Portable Solar...,- High-efficiency solar panels for rapid charg...


In [3]:
# Create a connection to an SQLite database file
conn = sqlite3.connect('./data/product_sqlite.db')

# Write the DataFrame to a table in the SQLite database
data.to_sql('product_table', conn, if_exists='replace', index=False)

12

In [4]:
# SQLITE

result = conn.execute("SELECT * FROM product_table limit 2").fetchall()

# Process the result
for row in result:
    rprint(row)

(
    'Ultimate Wireless Bluetooth Earbuds',
    79.99,
    4.4,
    "Elevate your music experience with our Ultimate Wireless Bluetooth Earbuds. These earbuds are designed for 
audiophiles who demand premium sound quality and convenience. With advanced Bluetooth 5.0 technology, you'll enjoy 
seamless connectivity and crystal-clear audio.",
    '- High-fidelity sound with deep bass and clear treble. - Noise-canceling technology for an immersive listening
experience. - 20 hours of playtime on a single charge. - Ergonomic design for a secure and comfortable fit. - Touch
controls for easy playback and calls. - IPX7 waterproof rating for workouts and outdoor activities. - Built-in 
microphone for hands-free calls.'
)

(
    'SmartHome Security Camera System',
    199.99,
    4.6,
    "Keep your home safe and secure with our SmartHome Security Camera System. This comprehensive package includes 
everything you need to monitor your property, inside and out. Whether you're at home or on the go, you can access 
live video feeds and receive instant alerts on your smartphone.",
    '- 1080p HD cameras for crystal-clear video quality. - Night vision technology for 24/7 surveillance. - Motion 
detection with customizable alert zones. - Two-way audio for communication with visitors. - Smartphone app for 
remote monitoring and control. - Expandable system with additional cameras and storage options. - Cloud storage and
local backup for video footage. - Easy DIY installation with mounting hardware included.'
)

In [5]:
from llama_index import (
    SQLDatabase,
    SimpleDirectoryReader,
    Document,
    StorageContext,
)
from llama_index.indices.struct_store import (
    NLSQLTableQueryEngine,
    SQLTableRetrieverQueryEngine,
)
from IPython.display import Markdown, display

## Create SQLDatabase Object

In [6]:
from llama_index import SQLDatabase
from sqlalchemy import engine, MetaData, create_engine, text, select

In [7]:
# SQLITE
engine = create_engine("sqlite:///" +"./data/product_sqlite.db")
connection = engine.connect()
result = connection.execute(text("SELECT * FROM product_table WHERE Price = 79.99"))
for row in result:
    rprint(row)


('Ultimate Wireless Bluetooth Earbuds', 79.99, 4.4, "Elevate your music experience with our Ultimate Wireless 
Bluetooth Earbuds. These earbuds are designed for audiophiles who demand premium sound quality and convenience. 
With advanced Bluetooth 5.0 technology, you'll enjoy seamless connectivity and crystal-clear audio.", '- 
High-fidelity sound with deep bass and clear treble. - Noise-canceling technology for an immersive listening 
experience. - 20 hours of playtime on  ... (72 characters truncated) ... Touch controls for easy playback and 
calls. - IPX7 waterproof rating for workouts and outdoor activities. - Built-in microphone for hands-free calls.')

('Portable Solar Charger', 79.99, 4.0, 'Stay charged on the go with our Portable Solar Charger. Harness the power 
of the sun to keep your devices powered wherever you are.', '- High-efficiency solar panels for rapid charging.<br>
- Built-in USB ports compatible with various devices.<br> - Compact and foldable design for easy carrying.<br> - 
Built-in battery for storing solar energy.<br> - LED indicators show charging status.<br> - Water-resistant for 
outdoor use.')

## Define SQL Database

In [10]:
from llama_index import SQLDatabase, ServiceContext
from llama_index.llms import OpenAI

In [11]:
llm = OpenAI(temperature=0.1, model = "gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm =llm)
sql_database = SQLDatabase(engine, include_tables=["product_table"])

## Text to SQL Query Engine

In [12]:
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine

In [13]:
query_engine = NLSQLTableQueryEngine(
    sql_database = sql_database,
    tables = ["product_table"])
query_str = "Which product was sold for 79.99?"
response = query_engine.query(query_str)
rprint(response)

Response(
    response='The products that were sold for 79.99 are the Ultimate Wireless Bluetooth Earbuds and the Portable 
Solar Charger.',
    source_nodes=[
        NodeWithScore(
            node=TextNode(
                id_='5acd51a8-91ff-40a0-8e93-db3f0907a866',
                embedding=None,
                metadata={},
                excluded_embed_metadata_keys=[],
                excluded_llm_metadata_keys=[],
                relationships={},
                text="[('Ultimate Wireless Bluetooth Earbuds',), ('Portable Solar Charger',)]",
                start_char_idx=None,
                end_char_idx=None,
                text_template='{metadata_str}\n\n{content}',
                metadata_template='{key}: {value}',
                metadata_seperator='\n'
            ),
            score=None
        )
    ],
    metadata={
        '5acd51a8-91ff-40a0-8e93-db3f0907a866': {},
        'sql_query': 'SELECT Product_Name\nFROM product_table\nWHERE Price = 79.99;',
        'result': [('Ultimate Wireless Bluetooth Earbuds',), ('Portable Solar Charger',)],
        'col_keys': ['Product_Name']
    }
)

## Part 2: Query-Time Retrieval of Tables for Text-to-SQL
If we don’t know ahead of time which table we would like to use, and the total size of the table schema overflows your context window size, we should store the table schema in an index so that during query time we can retrieve the right schema.

The way we can do this is using the SQLTableNodeMapping object, which takes in a SQLDatabase and produces a Node object for each SQLTableSchema object passed into the ObjectIndex constructor.

In [14]:
from llama_index.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine,
)
from llama_index.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index import VectorStoreIndex

# set Logging to DEBUG for more detailed outputs
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="product_table"))
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=1)
)

In [17]:
query_str = "What is the Rating for the product whose name is Portable Solar Charger?"
response = query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

<b>The rating for the product named Portable Solar Charger is 4.0.</b>

In [16]:
# you can also fetch the raw result from SQLAlchemy!
response.metadata["result"]

[('Ultimate Wireless Bluetooth Earbuds',), ('Portable Solar Charger',)]

In [19]:
import csv
# Define the column we want to embed vs which ones we want to store as metadat
columns_to_embed = ["Description", "Features"]
columns_to_metadata = ["Product Name", "Price", "Rating", "Description", "Features"]

In [20]:
## With CSV
# docs = []
# with open(filepath, newline="", encoding='utf-8-sig') as csvfile:
#     csv_reader = csv.DictReader(csvfile)
#     # ic(csv_reader)
#     for i, row in enumerate(csv_reader):
#         to_metadata = {col: row[col] for col in columns_to_metadata if col in row} 
#         # ic(to_metadata)
#         values_to_embed = {k: row[k] for k in columns_to_embed if k in row}
#         # ic(values_to_embed)
#         to_embed = '\n'.join(f"{k.strip()}: {v.strip()}" for k, v in values_to_embed.items())
#         # ic(to_embed)
#         newDoc = Document(text = to_embed, metadata = to_metadata)
#         docs.append(newDoc)

In [29]:
## With PANDAS DATAFRAME
# Assuming you have already loaded the data into a Pandas DataFrame df
columns_to_embed = ["Description", "Features"]
columns_to_metadata = ["Product Name", "Price", "Rating", "Description", "Features"]

docs = []
for i, row in data.iterrows():
    to_metadata = {col: row[col] for col in columns_to_metadata if col in row}
    values_to_embed = {k: row[k] for k in columns_to_embed if k in row}
    to_embed = '\n'.join(f"{k.strip()}: {v.strip()}" for k, v in values_to_embed.items())
    newDoc = Document(text=to_embed, metadata=to_metadata)
    docs.append(newDoc)

In [42]:
rprint(len(docs))

12

In [32]:
from llama_index.vector_stores import WeaviateVectorStore

import weaviate
import openai

from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())
import os
openai.api_key = os.getenv("OPENA_AI_KEY")


client = weaviate.Client(
    embedded_options=weaviate.embedded.EmbeddedOptions()
)

Started /home/zeus/.cache: process ID 71938


{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-01-28T06:19:13Z"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-01-28T06:19:13Z"}
{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-01-28T06:19:13Z"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2024-01-28T06:19:13Z"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50060","time":"2024-01-28T06:19:13Z"}
{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://127.0.0.1:8079","time":"202

{"level":"info","msg":"Completed loading shard blogpost_aoHZA7ipG29I in 9.273679ms","time":"2024-01-28T06:19:14Z"}
{"action":"hnsw_vector_cache_prefill","count":3000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-01-28T06:19:14Z","took":469200}


## Create Weaviate Schema

In [38]:
# Create Schema
podcast_schema = {
   "classes": [
       {
           "class": "Product",
           "description": "Listing of Products",
           "vectorizer": "text2vec-openai",
           "properties": [
               {
                  "name": "Product_Name",
                  "dataType": ["text"],
                  "description": "Name of the product.",
               },
               {
                  "name": "Price",
                  "dataType": ["number"],
                  "description": "Price of the product.",
               },
               {
                  "name": "Rating",
                  "dataType": ["number"],
                  "description": "Rating of the product.",
               },
               {
                  "name": "Description",
                  "dataType": ["text"],
                  "description": "Description of the product.",
               },
               {
                  "name": "Features",
                  "dataType": ["text"],
                  "description": "Feature of the product.",
               }
            ]
        }
    ]
}

client.schema.create(podcast_schema)
print("Product schema was created.")

Product schema was created.


{"level":"info","msg":"Created shard product_BkCPV0cszQYS in 2.92703ms","time":"2024-01-28T06:25:29Z"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-01-28T06:25:29Z","took":106122}


In [43]:
# Create a single document from a list of Documents
document = Document(text="\n\n".join([doc.text for doc in docs]))

In [45]:
rprint(document)

Document(
    id_='bd58a1b5-69a9-4d5e-84cb-d4ab9c4c93b7',
    embedding=None,
    metadata={},
    excluded_embed_metadata_keys=[],
    excluded_llm_metadata_keys=[],
    relationships={},
    text="Description: Elevate your music experience with our Ultimate Wireless Bluetooth Earbuds. These earbuds 
are designed for audiophiles who demand premium sound quality and convenience. With advanced Bluetooth 5.0 
technology, you'll enjoy seamless connectivity and crystal-clear audio.\nFeatures: - High-fidelity sound with deep 
bass and clear treble. - Noise-canceling technology for an immersive listening experience. - 20 hours of playtime 
on a single charge. - Ergonomic design for a secure and comfortable fit. - Touch controls for easy playback and 
calls. - IPX7 waterproof rating for workouts and outdoor activities. - Built-in microphone for hands-free 
calls.\n\nDescription: Keep your home safe and secure with our SmartHome Security Camera System. This comprehensive
package includes everything you need to monitor your property, inside and out. Whether you're at home or on the go,
you can access live video feeds and receive instant alerts on your smartphone.\nFeatures: - 1080p HD cameras for 
crystal-clear video quality. - Night vision technology for 24/7 surveillance. - Motion detection with customizable 
alert zones. - Two-way audio for communication with visitors. - Smartphone app for remote monitoring and control. -
Expandable system with additional cameras and storage options. - Cloud storage and local backup for video footage. 
- Easy DIY installation with mounting hardware included.\n\nDescription: Brew barista-quality coffee at home with 
our Professional Espresso Machine. This machine features a 15-bar pump for rich extraction and a steam wand for 
frothing milk to perfection.\nFeatures: - 15-bar pump for optimal espresso extraction.<br> - Steam wand for milk 
frothing.<br> - Durable stainless steel construction.<br> - Single and double shot filter baskets included.<br> - 
Cup warmer on top of the machine.<br> - Removable water reservoir for easy cleaning.\n\nDescription: Stay charged 
on the go with our Portable Solar Charger. Harness the power of the sun to keep your devices powered wherever you 
are.\nFeatures: - High-efficiency solar panels for rapid charging.<br> - Built-in USB ports compatible with various
devices.<br> - Compact and foldable design for easy carrying.<br> - Built-in battery for storing solar energy.<br> 
- LED indicators show charging status.<br> - Water-resistant for outdoor use.\n\nDescription: Elevate your style 
with our Luxury Leather Wallet. Handcrafted from premium leather, it offers ample space for cards, cash, and 
more.\nFeatures: - Handcrafted from genuine leather.<br> - Multiple card slots and compartments.<br> - RFID 
blocking technology for enhanced security.<br> - Sleek and slim design for easy carrying.<br> - Available in 
various colors.\n\nDescription: Achieve your fitness goals with our Fitness Tracker Smartwatch. Monitor your 
activity, heart rate, and receive notifications on your wrist.\nFeatures: - Heart rate monitor and activity 
tracking.<br> - Built-in GPS for accurate workout tracking.<br> - Sleep analysis and guided breathing 
exercises.<br> - Receive notifications from your smartphone.<br> - Water-resistant for workouts and everyday 
use.<br> - Long battery life.\n\nDescription: Unleash your inner chef with our Culinary Knife Set. This set 
includes a range of high-quality knives for precise and effortless slicing and dicing.\nFeatures: - High-carbon 
stainless steel blades for sharpness and durability.<br> - Full tang construction for balance and control.<br> - 
Ergonomic handles for comfortable grip.<br> - Knife block for safe storage.<br> - Set includes chef's knife, paring
knife, utility knife, and more.\n\nDescription: Say goodbye to tangled cables with our Wireless Charging Pad. 
Simply place your device on the pad for convenient and fast wireless charging.\nFeature

## Build Weaviate Index

In [47]:
from llama_index import VectorStoreIndex


In [50]:
vector_store = WeaviateVectorStore(weaviate_client=client, class_prefix="Product_index")
storage_context = StorageContext.from_defaults(vector_store=vector_store)


product_index = VectorStoreIndex.from_documents([document], storage_context=storage_context)


In [53]:
# set up text2SQL prompt
sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["product_table"],
)

## Build Query Engine

In [54]:

vector_query_engine = product_index.as_query_engine()

## Tell llama index what tool to use

In [55]:
from llama_index.tools.query_engine import QueryEngineTool

sql_tool = QueryEngineTool.from_defaults(
    query_engine = sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over a table containing: "
        "produtc listing, containing the price , rating, descriptions, and features of each product"
    ),
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for answering semantic questions about product listing",
)

## Create a router


In [56]:
from llama_index.query_engine.router_query_engine import RouterQueryEngine
from llama_index.selectors.llm_selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=([sql_tool] + [vector_tool]),
)

## Queries


In [57]:
response = query_engine.query("What is the Rating for the product whose name is Portable Solar Charger?")
rprint(str(response))

The rating for the product named Portable Solar Charger is 4.0.

In [59]:
response = query_engine.query("Tell me about Wireless Bluetooth Earbuds products in the product listing ")
rprint(str(response))

The Ultimate Wireless Bluetooth Earbuds are a top choice for audiophiles who value both sound quality and 
convenience. With advanced Bluetooth 5.0 technology, these earbuds offer seamless connectivity and crystal-clear 
audio. They provide high-fidelity sound with deep bass and clear treble, and also feature noise-canceling 
technology for an immersive listening experience. With 20 hours of playtime on a single charge, you can enjoy your 
music for extended periods of time. The ergonomic design ensures a secure and comfortable fit, and the touch 
controls make it easy to control playback and calls. Additionally, these earbuds have an IPX7 waterproof rating, 
making them suitable for use during workouts or in rainy conditions.

In [60]:
rprint(response)

Response(
    response='The Ultimate Wireless Bluetooth Earbuds are a top choice for audiophiles who value both sound quality
and convenience. With advanced Bluetooth 5.0 technology, these earbuds offer seamless connectivity and 
crystal-clear audio. They provide high-fidelity sound with deep bass and clear treble, and also feature 
noise-canceling technology for an immersive listening experience. With 20 hours of playtime on a single charge, you
can enjoy your music for extended periods of time. The ergonomic design ensures a secure and comfortable fit, and 
the touch controls make it easy to control playback and calls. Additionally, these earbuds have an IPX7 waterproof 
rating, making them suitable for use during workouts or in rainy conditions.',
    source_nodes=[
        NodeWithScore(
            node=TextNode(
                id_='c9a7926b-12bd-4182-9fc9-e229fe405718',
                embedding=None,
                metadata={},
                excluded_embed_metadata_keys=[],
                excluded_llm_metadata_keys=[],
                relationships={},
                text='[(\'Ultimate Wireless Bluetooth Earbuds\', 79.99, 4.4, "Elevate your music experience with 
our Ultimate Wireless Bluetooth Earbuds. These earbuds are designed for audiophiles who demand premium sound 
quality and convenience. With advanced Bluetooth 5.0 technology, you\'ll enjoy seamless connectivity and 
crystal-clear audio.", \'- High-fidelity sound with deep bass and clear treble. - Noise-canceling technology for an
immersive listening experience. - 20 hours of playtime on a single charge. - Ergonomic design for a secure and 
comfortable fit. - Touch controls for easy playback and calls. - IPX7 waterproof rating for...\')]',
                start_char_idx=None,
                end_char_idx=None,
                text_template='{metadata_str}\n\n{content}',
                metadata_template='{key}: {value}',
                metadata_seperator='\n'
            ),
            score=None
        )
    ],
    metadata={
        'c9a7926b-12bd-4182-9fc9-e229fe405718': {},
        'sql_query': "SELECT Product_Name, Price, Rating, Description, Features\nFROM product_table\nWHERE 
Product_Name LIKE '%Wireless Bluetooth Earbuds%'",
        'result': [
            (
                'Ultimate Wireless Bluetooth Earbuds',
                79.99,
                4.4,
                "Elevate your music experience with our Ultimate Wireless Bluetooth Earbuds. These earbuds are 
designed for audiophiles who demand premium sound quality and convenience. With advanced Bluetooth 5.0 technology, 
you'll enjoy seamless connectivity and crystal-clear audio.",
                '- High-fidelity sound with deep bass and clear treble. - Noise-canceling technology for an 
immersive listening experience. - 20 hours of playtime on a single charge. - Ergonomic design for a secure and 
comfortable fit. - Touch controls for easy playback and calls. - IPX7 waterproof rating for...'
            )
        ],
        'col_keys': ['Product_Name', 'Price', 'Rating', 'Description', 'Features'],
        'selector_result': MultiSelection(
            selections=[
                SingleSelection(
                    index=0,
                    reason='Choice 1 is most relevant because it mentions translating a natural language query into
a SQL query over a table containing product listings, which would be useful for retrieving information about 
Wireless Bluetooth Earbuds products in the product listing.'
                )
            ]
        )
    }
)

In [61]:
response = query_engine.query("How many product cost 79.99 in the product listing?")
rprint(str(response))

There are 2 products in the product listing that cost 79.99.

In [62]:
rprint(response)

Response(
    response='There are 2 products in the product listing that cost 79.99.',
    source_nodes=[
        NodeWithScore(
            node=TextNode(
                id_='19c0746f-afca-4ea8-86cc-687554e9019b',
                embedding=None,
                metadata={},
                excluded_embed_metadata_keys=[],
                excluded_llm_metadata_keys=[],
                relationships={},
                text='[(2,)]',
                start_char_idx=None,
                end_char_idx=None,
                text_template='{metadata_str}\n\n{content}',
                metadata_template='{key}: {value}',
                metadata_seperator='\n'
            ),
            score=None
        )
    ],
    metadata={
        '19c0746f-afca-4ea8-86cc-687554e9019b': {},
        'sql_query': 'SELECT COUNT(*) FROM product_table WHERE Price = 79.99',
        'result': [(2,)],
        'col_keys': ['COUNT(*)'],
        'selector_result': MultiSelection(
            selections=[
                SingleSelection(
                    index=0,
                    reason='Choice 1 is most relevant because it mentions translating a natural language query into
a SQL query over a table containing product listings, which would be necessary to find the number of products that 
cost 79.99 in the product listing.'
                )
            ]
        )
    }
)

In [63]:
response = query_engine.query("How many product are similar to Wireless Bluetooth Earbuds products?")
rprint(str(response))

There is 1 product that is similar to Wireless Bluetooth Earbuds products.

In [64]:
rprint(response)

Response(
    response='There is 1 product that is similar to Wireless Bluetooth Earbuds products.',
    source_nodes=[
        NodeWithScore(
            node=TextNode(
                id_='eb748814-31ee-48c5-b03e-63c397f58e61',
                embedding=None,
                metadata={},
                excluded_embed_metadata_keys=[],
                excluded_llm_metadata_keys=[],
                relationships={},
                text='[(1,)]',
                start_char_idx=None,
                end_char_idx=None,
                text_template='{metadata_str}\n\n{content}',
                metadata_template='{key}: {value}',
                metadata_seperator='\n'
            ),
            score=None
        )
    ],
    metadata={
        'eb748814-31ee-48c5-b03e-63c397f58e61': {},
        'sql_query': "SELECT COUNT(*) FROM product_table WHERE Product_Name LIKE '%Wireless Bluetooth Earbuds%'",
        'result': [(1,)],
        'col_keys': ['COUNT(*)'],
        'selector_result': MultiSelection(
            selections=[
                SingleSelection(
                    index=0,
                    reason='Choice 1 is most relevant because it mentions a table containing product listings, 
which would likely include information about similar products such as Wireless Bluetooth Earbuds.'
                )
            ]
        )
    }
)

In [65]:
response = query_engine.query("list the products whose descriptions are similar.")
rprint(str(response))

The products whose descriptions are similar are:
1. Organic Bamboo Bed Sheets
2. Virtual Reality Headset

In [67]:
rprint(response.metadata)

{
    'a57285ab-1c52-48f8-b669-54b224d598ad': {},
    'dd13f5be-f2f6-4a70-b076-073bf9ec2f58': {},
    'selector_result': MultiSelection(
        selections=[
            SingleSelection(
                index=1,
                reason='Choice 2 is most relevant because it specifically mentions answering semantic questions 
about product listing, which includes determining similarity between product descriptions.'
            )
        ]
    )
}

In [77]:
rprint(data.loc[data.Product_Name.isin(('Organic Bamboo Bed Sheets', 'Virtual Reality Headset')), "Description"].values)

['Experience ultimate comfort with our Organic Bamboo Bed Sheets. These luxuriously soft sheets are made from 
sustainable and breathable bamboo fibers.'
 'Immerse yourself in virtual worlds with our Virtual Reality Headset. Experience gaming, entertainment, and 
exploration like never before.']

In [83]:
response = query_engine.query("Which product has 'heart rate' in its description")
rprint(str(response))

The product that has 'heart rate' in its description is the Fitness Tracker Smartwatch.

In [84]:
rprint(response)

Response(
    response="The product that has 'heart rate' in its description is the Fitness Tracker Smartwatch.",
    source_nodes=[
        NodeWithScore(
            node=TextNode(
                id_='0dbf45a2-3a3c-497b-bb2e-d57eca866a37',
                embedding=None,
                metadata={},
                excluded_embed_metadata_keys=[],
                excluded_llm_metadata_keys=[],
                relationships={},
                text="[('Fitness Tracker Smartwatch',)]",
                start_char_idx=None,
                end_char_idx=None,
                text_template='{metadata_str}\n\n{content}',
                metadata_template='{key}: {value}',
                metadata_seperator='\n'
            ),
            score=None
        )
    ],
    metadata={
        '0dbf45a2-3a3c-497b-bb2e-d57eca866a37': {},
        'sql_query': "SELECT Product_Name\nFROM product_table\nWHERE Description LIKE '%heart rate%'",
        'result': [('Fitness Tracker Smartwatch',)],
        'col_keys': ['Product_Name'],
        'selector_result': MultiSelection(
            selections=[
                SingleSelection(
                    index=0,
                    reason='Choice 1 is most relevant because it mentions translating a natural language query into
a SQL query over a table containing product listings, which would include descriptions of each product. This 
suggests that it would be able to handle a query about the description of a product.'
                )
            ]
        )
    }
)

In [85]:
response = query_engine.query("What is the sum of all prices for all products whose price is $79.99?")
rprint(str(response))

The sum of all prices for all products whose price is $79.99 is $159.98.

In [87]:
rprint(response.metadata)

{
    'f419eef1-48cd-4084-a103-47d7dd9f3bc0': {},
    'sql_query': 'SELECT SUM(Price) FROM product_table WHERE Price = 79.99;',
    'result': [(159.98,)],
    'col_keys': ['SUM(Price)'],
    'selector_result': MultiSelection(
        selections=[
            SingleSelection(
                index=0,
                reason='Choice 1 is the most relevant because it mentions translating a natural language query into
a SQL query over a table containing product listings, including the price. This suggests that it can be used to 
find the sum of all prices for all products.'
            )
        ]
    )
}